# Allocation Agent: Monte Carlo Tree Search (MCTS)

A full review of MCTS is available in [Browne et al., 2012](http://ccg.doc.gold.ac.uk/ccg_old/papers/browne_tciaig12_1.pdf), and has been the basis of the below work.

Broadly speaking MCTS randomly explores a decision space, building a tree of connected moves and their associated rewards. Over multiple iterations the MCTS determines which decisions are most likely to result in a positive outcome. Ultimately the best decision is the one that provides the best long term reward, the definition of which depends on the specific domain. For example, when creating a MCTS agent to play a board game, the long term reward would be a 0 or 1 depending on whether you won or lost the game after making the current move. 

In the context of bed allocation we do not have a natural end state. Therefore the long term reward is determined as the total reward incurred after N time has passed according to the equation below. Here $R_{n}$ represents a reward associated with  the state of the hospital at a given time step, $\gamma \epsilon [0, 1]$ is the discount factor. The reward associated with a hospital state is $1 - total penalties$ incurred. The first term in the equation, $R_{1}$, is the immediate reward associated with the current allocation i.e., the greedy allocation score, and the subsequent terms are rewards associated with future states, where the discount factor weighs the relative importance of these future states against the current state. 
$$
\begin{align}
R = R_{1} + \gamma R_{2} +\gamma^{2}R_{3} ... +\gamma^{N}R_{N}
\end{align}
$$

In this notebook we demonstrate how to run the MCTS allocation algorithm in a simplified virtual hospital; see `src/agent/mcts` and `src/agent/simulation` for the relevant code. More details of the implementation are provided at the end of this notebook.

## 1. Import required modules

_Note:_ you will need to first install the module as per the instructions in the main README, and run a notebook server from within the same virtual environment to have access to the `hospital` submodules.

In [ ]:
import cloudpickle
import copy
import time
import random

import warnings
warnings.filterwarnings('ignore')

from tqdm.notebook import tqdm

import agent.utils as utils
import agent.policy as policy
import agent.run_mcts as mcts
from hospital.people import Patient
from hospital.building import Hospital, MedicalWard, SurgicalWard, Room
from hospital.equipment.bed import Bed
import hospital.restrictions.ward as R

## 2. Hospital Environment

The MCTS implementation can take a long time to run, and have high memory requirements. To demonstrate how it can be run we create a simplified scenario with a toy hospital containing 2 wards (1 medical, 1 surgical), and 5 beds each. The Medical ward will have a restriction for not allowing surgical patients and vice versa. 

In [ ]:
beds = [Bed(name=f"B00{i}") for i in range(10)]
wards = [
    MedicalWard(name="MedicalWard", rooms=[Room(name="R000", beds=beds[:5])]),
    SurgicalWard(name="SurgicalWard", rooms=[Room(name="R001", beds=beds[5:])]),
]
h = Hospital(name="Hospital", wards=wards)

# Add ward restrictions
h.wards[0].restrictions = [R.NoSurgical(10)]
h.wards[1].restrictions = [R.NoMedical(5)]

h.render()

In [ ]:
# you will also need to normalise the penalties to lie between 0 and 1
utils.normalise_ward_penalties(h)

## 3. Create patients to admit to the hospital

To generate realistic patients utilise the PatientSampler class which takes a day of week and an hour of the day and returns a forecast for the number of patients estimated to arrive each hour, `N`. By default, the patients are synthesied with random data where the distribution for each attributes is informed by aggregated historic data. If historic patient data is available then a pool of historic patients can be saved and setting `historic=True` will return more accurate marginal distributions accross all patient attributes. Otherwise, the returned number of patients `N` is a random number, and the returned patient attributes are randomly generated. See `src/forecasting/patient_sampler`. 

```
sampler = PatientSampler("monday", 9)
forecast_window=2
forecasted_patients = sampler.sample_patients(forecast_window=forecast_window, num_samples=1)

# we can unpack the above structure into a list of lists
# each sublist represents an hour of patients
arrivals = []
for _, patients in forecasted_patients[0].items():
     arrivals.append(patients)
```

We will instead use the code below to create a simplified list of arriving patients, by initialising patients with the default value for most fields, such that there won't be any additional patient level restrictions (e.g., patient needs side room for immunosuppression). The arrivals list will still have the same structure as above. 

In [ ]:
def generate_random_name() -> str:
    """
    2 random letters + 2 random numbers. As a string.
    """
    letters = "abcdefghijklmnopqrstuvwxyz"
    digits = "0123456789"
    characters = random.choices(letters, k=2) + random.choices(digits, k=2)
    name = "".join(characters)
    return name

def generate_simple_patient() -> Patient:
    """Returns a random patient, without any patient level restrictions."""
    
    # department and specialty
    department = ["medicine", "surgery"][utils.bernoulli(0.5)]

    # Patient, all other attributes are default=False
    patient = Patient(
        name=generate_random_name(),
        sex=["male", "female"][utils.bernoulli(0.5)],
        department=department
    )
    
    return patient

def generate_arrivals(max_per_hour: int, forecast_window: int) -> list:
    """Creates a list of arriving patients for each hour of in the forecast window"""
    arrivals =[]
    for hours in range(forecast_window):
        arrivals.append([generate_simple_patient() for _ in range(random.randint(0, max_per_hour))])
                        
    return arrivals
    

In [ ]:
# patient currently being allocated
patient = Patient(
    name="patient_0",
    sex="female",
    department="medicine",
)

# 'forecasted' arrivals
arrivals = generate_arrivals(3, 4)

print(f"Incoming patients per hour: {[len(l) for l in arrivals]}")

# now we insert the patient we are currently trying to 
# allocate as the first patient, as time t=1.
arrivals = [[patient]] + arrivals

Now we can run the MCTS as below. The output in `mcts_output` is a list of dictionaries where each dictionary represents a possible allocation for patient_0, and the results of the tree search for that option, further details of the tree search are provided below. The items of each dictionary are:
- `action`, the allocation of the current patient into a bed. It is possible to consider allocating multiple patients in a timestep, as such, the action is represented by a dictionary of bed_name:patient pairs, for each patient at t=0 in the tree search. In our example there is just one patient.
- `score`, the immediate pentaly associated with allocating the current patient(s) to the suggested bed(s).
- `violated_restrictions`, the set of restrictions (if any), violated by assigning the patient(s) to the suggested bed(s).
- `ucb_score`, the tree policy score associated with the suggested allocation. See below for more details on UCB score. 
- `visit_count`, the number of times the node representing the suggested allocation was visited during tree search. 

There are several potential strategies for determining what the best allocation option is. The one with the lowest `score` is equivalent to making a greedy optimisation that selects the best bed given the current circumstances of the hospital; whereas, making a choice between the highest `ucb_score`, or `visit_count` or a weighted average of both provides the best allocation according to the MCTS. 

In [ ]:
h_copy = copy.deepcopy(h)
t = time.time()
mcts_node = mcts.run_mcts(
        h_copy,
        arrivals,
        discount_factor=0.9,
        n_iterations=100,
)
elapsed = time.time() - t
mcts_output = mcts.construct_mcts_output(h_copy, mcts_node, patient)

In [ ]:
print(f"Time taken to compute suggestions: {round(elapsed,2)}s")
mcts_output

# 4. Details on implementation

Below we describe the four stages of the MCTS algorithm as they are specifically implemented for the bed allocation agent. The implementation utilises the `anytree` library to build a tree structure, where each node represents a specific state of the hospital and each level of the tree represents a time step. Time steps are incremented in hours and connected to the number of forecasted admissions arriving each hour. The input to the treesearch is a queue of patients arriving at each time step, with the current patient to be allocated (`t=0`) as the first entry in this queue, and the current state of the hospital as the root node to search from.

<ol>
<li><b>Selection</b></li>

Starting at the root node a child node is selected. The root node represents the current state of the hospital at time $t=0$ where the state is defined by the patients that are currently occupying beds and the set of empty beds. 

In the first iteration of the tree search, the algorithm selects the root node and moves to the expansion step. In subsequent iterations it will traverse from the root and choose one of the child nodes according to the tree policy. The <i>tree policy</i> is the UCB score, in which $\overline{R}$ is the mean reward from visiting that node, $N_{pi}$ is the number of times its parent node was visited and $N_{i}$ is the number of times the node itself has been visited. The first term encourages the algorithm to select nodes that have previously resulted in good outcomes, while the second encourages the algorithm to explore options that it hasn’t visited as often.
$$
\begin{align}
UCB = \overline{R} + \sqrt{\frac{2 \log{N_{p}}}{N_{i}}}
\end{align}
$$
<li><b>Expansion</b></li>
Once a node is selected, a child node is attached to represent each possible decision state for that time step.

For example, we have a hospital with 4 beds, 2 are occupied and 2 are available. We are currently trying to allocate a patient P1. As there are two possible decisions, two nodes can be attached that represent 1) allocating P1 to the first available bed and 2) allocation P1 to the second available bed.

As we progress through the tree search, we may encounter time steps where multiple patients have arrived. In such cases, a node is expanded for each possible combination of patients to available beds.
<li><b>Simulation</b></li>
From one of the attached children we then simulate a future. The simulation stage involves the following steps:
<ul>
<li>Each patient currently within the hospital has a length of stay attribute, and an expected length of stay attribute. At the start of the simulation step, the length of stay counters are incremented by one.</li>
<li>Then a discharge model is applied to discharge existing patients. The probability of being discharged increases according to proximity to your expected length of stay.</li>
<li>The patients arriving in the given time-step are then assigned to beds according to the default policy. The default policy is a random assignment of patients to beds to available beds.</li>
</ul>
<li><b>Backpropagation</b></li>
The total penalty of the hospital is calculated after the simulation step. This is the sum of all penalties for each broken restriction within the hospital. We then backpropagate this score up the tree to distribute the outcome across all decisions along the currently explored path. 

This stage updates the UCB score (tree policy score) and visit count for each node that was traversed along the current decision pathway. If the result of the simulation was good, the UCB scores of each node will have increased, making it more likely that future iterations of the tree search will select these nodes again. The reverse is also true. In this manner MCTS is more tractable than a completely random search of the possible decision pathways as it more frequently visits the most promising options during the selection stage.
</ol>
The above procedure is repeated multiple times until a maximum number of iterations have been reached. At this point the tree object is returned and the best child node of the root is selected as the optimal allocation for the current patient. There are several potential strategies for determining what the best node is. In the current implementation we choose the node that has the highest visit count. Alternative approaches such as choosing the node with the highest UCB score or some balance of the two, and how this affects outcomes, remain to be explored in future work.

### Limitations
In the above implementation we take a single sample from the demand forecast and use this as a fixed version of the future. This means that the future within the treesearch is deterministic and significantly reduces the search space, and branching of the tree, allowing the algorithm to find a recommendation in a more tractable timeframe. However, a single sample from the forecast represents one of the possible futures. To truly capture the variability of incoming patients, we envisage a strategy where multiple simultaneous tree searches are implemented, each using a separate sample from the forecasted admissions. These could be run in parallel to increase runtime efficiency, and the final suggested allocation would be the bed that has the average highest ranking across the ensemble of tree searches. The efficacy of this strategy and alternative approaches to dealing with non deterministic search spaces remain to be explored. 

Despite fixing the set of arrivals within a tree-search we can still experience an intractable amount of branching that makes the current implementation of MCTS unsuitable for operational use. For example, if there are just 4 empty beds in the hospital, and 9 arriving patients within a time step, the tree expands into 840 possible permutations of patients to beds. With multiple time steps into the future this can compound and result in either memory issues or extremely long compute times. Further work is needed to explore engineering strategies that can make MCTS more operationally feasible.